# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
#found code here for opening zip files: https://stackoverflow.com/questions/26942476/reading-csv-zipped-files-in-python
import zipfile

orders = zipfile.ZipFile('./Orders.zip') 
orders = pd.read_csv(orders.open('Orders.csv'))
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
# your # your code here
# First sub problem: How to aggregate the amount_spent for unique customers
agg_sum_amount = orders.groupby(['CustomerID','Country']).agg({'amount_spent': sum}).reset_index()
agg_sum_amount

,CustomerID,Country,amount_spent
0,12346,United Kingdom,77183.60
1,12347,Iceland,4310.00
2,12348,Finland,1797.24
3,12349,Italy,1757.55
4,12350,Norway,334.40
...,...,...,...
4342,18280,United Kingdom,180.60
4343,18281,United Kingdom,80.82
4344,18282,United Kingdom,178.05
4345,18283,United Kingdom,2094.88


In [4]:
# second sub problem: How to select customers whose aggregated amount_spent is in a given quantile range
#The first group is VIP Customers whose aggregated expenses at your global chain stores are above the 95th percentile 
#(aka. 0.95 quantile).
#The second group is Preferred Customers whose aggregated expenses are between the 75th and 95th percentile.

#Calculcating the 0.95 quantile for amount_spent
quan95_spent = agg_sum_amount['amount_spent'].quantile(q=0.95)

#Calculcating the 0.75 quantile for amount_spent
quan75_spent = agg_sum_amount['amount_spent'].quantile(q=0.75)

#Getting only customers with amount_spent between 75th and 95th percentile:
cus_75_95 = agg_sum_amount[(agg_sum_amount['amount_spent'] > quan75_spent) & (agg_sum_amount['amount_spent'] < quan95_spent)]

#Getting only customers with amount_spent above 95th percentile:
cus_over_95 = agg_sum_amount[agg_sum_amount['amount_spent'] >= quan95_spent]
cus_over_95

,CustomerID,Country,amount_spent
0,12346,United Kingdom,77183.60
10,12357,Switzerland,6207.67
12,12359,Cyprus,6372.58
52,12409,Switzerland,11072.67
57,12415,Australia,124914.53
...,...,...,...
4215,18109,United Kingdom,8052.97
4237,18139,United Kingdom,8438.34
4261,18172,United Kingdom,7561.68
4300,18223,United Kingdom,6484.54


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [5]:
def labelCustomer(row):
    if row['amount_spent'] >=quan95_spent:
        return 'VIP'
    elif (row['amount_spent'] > quan95_spent) & (row['amount_spent'] < quan95_spent):
        return 'Preferred'
    else:
        return 'normal'

In [6]:
#Applying function to create new column for labeling customers:
agg_sum_amount['Labeling'] = agg_sum_amount.apply(labelCustomer, axis=1)
agg_sum_amount.head()

,CustomerID,Country,amount_spent,Labeling
0,12346,United Kingdom,77183.60,VIP
1,12347,Iceland,4310.00,normal
2,12348,Finland,1797.24,normal
3,12349,Italy,1757.55,normal
4,12350,Norway,334.40,normal


In [7]:
agg_sum_amount1=agg_sum_amount[agg_sum_amount['Labeling']=='VIP']
agg_sum_amount1.info

<bound method DataFrame.info of       CustomerID         Country  amount_spent Labeling
0          12346  United Kingdom      77183.60      VIP
10         12357     Switzerland       6207.67      VIP
12         12359          Cyprus       6372.58      VIP
52         12409     Switzerland      11072.67      VIP
57         12415       Australia     124914.53      VIP
...          ...             ...           ...      ...
4215       18109  United Kingdom       8052.97      VIP
4237       18139  United Kingdom       8438.34      VIP
4261       18172  United Kingdom       7561.68      VIP
4300       18223  United Kingdom       6484.54      VIP
4306       18229  United Kingdom       7276.90      VIP

[218 rows x 4 columns]>

In [8]:
table = pd.pivot_table(agg_sum_amount1, values='Labeling', index=["Country"],
                    columns=[], aggfunc="count")
table["Labeling"].sort_values(ascending=False)



Country
United Kingdom     178
Germany             11
France               9
Switzerland          3
Spain                2
Portugal             2
Japan                2
EIRE                 2
Finland              1
Channel Islands      1
Netherlands          1
Norway               1
Singapore            1
Denmark              1
Sweden               1
Cyprus               1
Australia            1
Name: Labeling, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [14]:
VIPpref = agg_sum_amount[(agg_sum_amount['Labeling'] == 'VIP') | (agg_sum_amount['Labeling'] == 'Preferred')]
VIPpref

,CustomerID,Country,amount_spent,Labeling
0,12346,United Kingdom,77183.60,VIP
10,12357,Switzerland,6207.67,VIP
12,12359,Cyprus,6372.58,VIP
52,12409,Switzerland,11072.67,VIP
57,12415,Australia,124914.53,VIP
...,...,...,...,...
4215,18109,United Kingdom,8052.97,VIP
4237,18139,United Kingdom,8438.34,VIP
4261,18172,United Kingdom,7561.68,VIP
4300,18223,United Kingdom,6484.54,VIP


In [16]:
table = pd.pivot_table(VIPpref, values=['Labeling'], index=['Country'], aggfunc="count")

table["Labeling"].sort_values(ascending=False)

Country
United Kingdom     178
Germany             11
France               9
Switzerland          3
Spain                2
Portugal             2
Japan                2
EIRE                 2
Finland              1
Channel Islands      1
Netherlands          1
Norway               1
Singapore            1
Denmark              1
Sweden               1
Cyprus               1
Australia            1
Name: Labeling, dtype: int64